In [5]:
#Imports
import pandas as pd
import altair as alt
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set_theme(style="ticks")
import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')

testClean = data
testClean["job"] = testClean["job"].replace(['unknown'], "admin.")
testClean["marital"] = testClean["marital"].replace(['unknown'], "married")
testClean["education"] = testClean["education"].replace(['unknown'], "university.degree")
testClean["default"] = testClean["default"].replace(['unknown'], "no")
testClean["housing"] = testClean["housing"].replace(['unknown'], "yes")
testClean["loan"] = testClean["loan"].replace(['unknown'], "no")
testClean['poutcome'] = testClean['poutcome'].replace(['nonexistent'], "failure")
testClean['pdays'] = testClean['pdays'].apply(lambda x: 0 if x == 999 else x)
testClean[['job','marital',"education",'default','housing','contact','month','day_of_week','poutcome','loan','y']] = testClean[['job','marital',"education",'default','housing','contact','month','day_of_week','poutcome','loan','y']].apply(lambda x: pd.factorize(x)[0])
testClean = testClean[~testClean['pdays'].isna()]
testClean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37069 entries, 0 to 37068
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             37069 non-null  int64  
 1   job             37069 non-null  int64  
 2   marital         37069 non-null  int64  
 3   education       37069 non-null  int64  
 4   default         37069 non-null  int64  
 5   housing         37069 non-null  int64  
 6   loan            37069 non-null  int64  
 7   contact         37069 non-null  int64  
 8   month           37069 non-null  int64  
 9   day_of_week     37069 non-null  int64  
 10  campaign        37069 non-null  int64  
 11  pdays           37069 non-null  int64  
 12  previous        37069 non-null  int64  
 13  poutcome        37069 non-null  int64  
 14  emp.var.rate    37069 non-null  float64
 15  cons.price.idx  37069 non-null  float64
 16  cons.conf.idx   37069 non-null  float64
 17  euribor3m       37069 non-null 

In [20]:
X = testClean.drop('y', axis=1)
y = testClean['y']
#X.head()

In [21]:
mutual_info = mutual_info_classif(X, y)
mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info.sort_values(ascending=False)

euribor3m         0.075186
cons.conf.idx     0.070493
cons.price.idx    0.070262
nr.employed       0.062504
emp.var.rate      0.057198
pdays             0.033277
poutcome          0.031846
month             0.027893
previous          0.019821
age               0.016104
contact           0.015389
job               0.011135
education         0.005506
campaign          0.004256
day_of_week       0.002999
default           0.002313
housing           0.001171
marital           0.001114
loan              0.000000
dtype: float64

In [27]:
sel_three_feat = SelectKBest(mutual_info_classif, k=4).fit(X, y)
sel_bool = sel_three_feat.get_support()
X_sel = X[X.columns[sel_bool]]
#X_sel.head()

In [33]:
X_main, X_test, y_main, y_test = train_test_split(X_sel, y, test_size=0.1 , random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.2, random_state=42)

In [34]:
dt = DecisionTreeClassifier(criterion="log_loss")
dt.fit(X_train, y_train)
y_pred = dt.predict(X_val)


print("Precision:",metrics.precision_score(y_val, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))
print("Classification Report:")
print(metrics.classification_report(y_val, y_pred))

Precision: 0.5478260869565217
Confusion Matrix:
[[5778  156]
 [ 550  189]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5934
           1       0.55      0.26      0.35       739

    accuracy                           0.89      6673
   macro avg       0.73      0.61      0.65      6673
weighted avg       0.87      0.89      0.88      6673



In [35]:
dt = DecisionTreeClassifier(criterion="log_loss")
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)


print("Precision:",metrics.precision_score(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(metrics.classification_report(y_test, y_pred))

Precision: 0.5960591133004927
Confusion Matrix:
[[3169   82]
 [ 335  121]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      3251
           1       0.60      0.27      0.37       456

    accuracy                           0.89      3707
   macro avg       0.75      0.62      0.65      3707
weighted avg       0.87      0.89      0.87      3707

